In [ ]:
from abc import ABC, abstractmethod


class Module(ABC):


    def __init__(self, id, stav):
        self._id = id
        self._stav = stav


    @property
    def id(self):
        return self._id


    @id.setter
    def id(self, value):
        self._id = value


    @property
    def stav(self):
        return self._stav


    @stav.setter
    def stav(self, value):
        self._stav = value


    @abstractmethod
    def atomicka_kontrola(self, other):
        pass


    def __rshift__(self, other): #self > other = modul zkontroluje jiny modul
        self.atomicka_kontrola(other)


    def __lshift__(self, other): # self < other = modul se necha zkontrolovat od jineho modulu
        other.atomicka_kontrola(self)

In [ ]:
from module import Module
from random import random


class PreparataModule(Module):


    def __init__(self, id, stav, pchybny_spravne):
        super().__init__(id, stav)
        self._pchybny_spravne = pchybny_spravne #pravdepodobnost, ze modul v chybnem stavu spravne odhali stav jineho modulu


    def atomicka_kontrola(self, other):
        if self.stav == 0:
            return other.stav
        else:
            return 1 if other.stav == 1 and random() <= self._pchybny_spravne else 0

In [ ]:
class System:


    def __init__(self, moduly = [], atomicke_kontroly = []):
        self._moduly = moduly
        self._atomicke_kontroly = atomicke_kontroly #{id_atomicke_kontroly:(id_kontrolujici, id_kontrolovany)}


    @property
    def moduly(self):
        return self._moduly


    @moduly.setter
    def moduly(self, value):
        self._moduly = value


    def matice_sousednosti(self):
        matice = [[0]*len(self._atomicke_kontroly) for i in range(len(self._atomicke_kontroly))]      
        for atomicka_kontrola in self._atomicke_kontroly:
            matice[atomicka_kontrola[0]][atomicka_kontrola[1]] = 1
        return matice    


    def ziskej_syndrom(self):
        syndrom = [[0]*len(self._atomicke_kontroly) for i in range(len(self._atomicke_kontroly))]
        for i in range(len(self._atomicke_kontroly)):
            for j in range(len(self._atomicke_kontroly)):
                if (i, j) in self._atomicke_kontroly:
                    syndrom[i][j] = self.moduly[i].atomicka_kontrola(self.moduly[j])
        return syndrom

In [ ]:
from preparata import PreparataModule
from system import System


def main():

    #tvorba modulu
    nmodulu = 4
    moduly = []
    for i in range(nmodulu):
        moduly.append(PreparataModule(i, 0, 0.5))
    moduly[2].stav = 1

    #tvorba atomickych kontrol
    atomicke_kontroly = [(0,1),(0,2),(1,3),(3,0)]

    #tvorba samodiagnostikovatelneho systemu
    print("stavy modulu:")
    system = System(moduly, atomicke_kontroly)
    for modul in system.moduly:
        print(modul.id, modul.stav)

    #vypis diagnostickeho grafu ze struktury systemu
    print("diagnosticky graf:")
    diagnosticky_graf = system.matice_sousednosti()
    for radek in diagnosticky_graf:
        print(radek)

    #vypis tabulky syndromu
    print("tabulka syndromu:")
    syndrom = system.ziskej_syndrom()
    for radek in syndrom:
        print(radek)


if __name__ == "__main__":
    main()